In [117]:
using Random
using LinearAlgebra
using SparseArrays
using PyCall
using Statistics

In [118]:
L = 2;
Number_Of_Noise = 4*L^2-6*L+13-3;
Random.seed!(7000)
NOISE = 2*rand(Float64,Number_Of_Noise).-1;

In [119]:
#Rx(theta) = exp(-1im*(theta*[-1 1;1 -1]/2));
#Rx(theta) = [cos(theta/2) -1im*sin(theta/2) ; -1im*sin(theta/2)  cos(theta/2)];
Rx(theta)=exp(-1im*(theta/2)*[1 1;1 1]);
int(x) = floor(Int,x);

In [120]:
#Rx(theta) = exp(-1im*theta*[1 1;1 1]/2);
#Rx(theta) = [cos(theta/2) -1im*sin(theta/2) ; -1im*sin(theta/2)  cos(theta/2)];#

#round.(-exp(-1im*pi*([1 1;1 1]/2)); digits = 3)

Ry(theta) = [cos(theta/2) -sin(theta/2) ; sin(theta/2) cos(theta/2)];

Pauli_X = [0 1;1 0];
Pauli_Y = [1 -1im;1im 0];
Pauli_Z = [1 0;0 -1];
X = [0 -1;-1 0];
I2 = [1 0; 0 1];
Z = [1 0;0 -1];
H = (1/sqrt(2))*[1 1;1 -1]
Hadamard(noise) = exp(-1im*(pi/2+noise)*(I2-H)) #Ry(pi/2+noise)*Pauli_Z;
CX(noise) = exp(-1im*((pi/2+noise))*[1 1;1 1]);

In [121]:
CX(0)

2×2 Matrix{ComplexF64}:
 1.1742e-16-1.23373e-16im         -1.0-1.66533e-16im
       -1.0-5.55112e-17im  3.53512e-17-2.10207e-16im

In [122]:
Rx(pi)

2×2 Matrix{ComplexF64}:
 1.1742e-16-1.23373e-16im         -1.0-1.66533e-16im
       -1.0-5.55112e-17im  3.53512e-17-2.10207e-16im

In [123]:
Hadamard(0.0)

2×2 Matrix{ComplexF64}:
 0.707107-5.55112e-17im   0.707107-1.38778e-17im
 0.707107+1.11022e-16im  -0.707107-2.77556e-16im

In [124]:
"""

Following function takes a 2x2 matrix (Gate) and qubit position (Qubit) and
returns the resultant matrix.

For example, the matrix for the gate U acting on the 3-rd qubit for N=5
qubit system is given by   I (x) I (x) U (x) I (x) I; where (x) is the
tensor product.

"""

function Matrix_Gate(Gate, Qubit) # Previously known as multi qubit gate.
    
    ## The case Qubit=1 is treated differently because we need to
    # initialize the matrix as U before starting the kronecker product.
    
    if Qubit == 1
        
        M = sparse(Gate)
        for i=2:L
            M = kron(M, sparse([1 0;0 1]))
        end
        
    else
        
        M = sparse([1 0;0 1])
        for i=2:L
            if i == Qubit
                M = kron(M, Gate)
            else
                M = kron(M, sparse([1 0;0 1]))
            end
        end
    end
    
    return M
end;

In [125]:
Identity(dimension) = 1* Matrix(I, dimension, dimension);

In [126]:
"""

The following function returns a controlled U gate matrix.

Input  : c (integer), t(integer), U (unitary operator).
Output : Matrix of the multicontrolled U gate with control qubit c and target qubit t.

"""

function CU(U,c,t)
    
    I2 = sparse([1 0;0 1])
    Z = sparse([1 0;0 -1])

    PI_0 = (I2+Z)/2
    PI_1 = (I2-Z)/2
     
    #function Rx(Noise)
        #A = cos((pi+Noise)/2)
        #B = -1im*sin((pi+Noise)/2)
        #return 1im*[A B;B A]
    #end
    
    Matrices = Dict("I" => I2,"PI_0" => PI_0,"U" => U, "PI_1" => PI_1)
    
    p0 = fill("I", L)
    p1 = fill("I", L)
    
    p0[c] = "PI_0"
    p1[c] = "PI_1"
    p1[t] = "U"

    
    PI_0_matrix = Matrices[p0[1]]
    for i = 2:L
        PI_0_matrix = kron(PI_0_matrix,Matrices[p0[i]])
    end        
        
    PI_1_matrix = Matrices[p1[1]]   
    for i = 2:L
        PI_1_matrix = kron(PI_1_matrix,Matrices[p1[i]])        
    end
           
    #return p0,p1
    return PI_0_matrix + PI_1_matrix     
end;

In [127]:
using PyCall
py"""
import numpy
import numpy.linalg
def adjoint(psi):
    return psi.conjugate().transpose()
def psi_to_rho(psi):
    return numpy.outer(psi,psi.conjugate())
def exp_val(psi, op):
    return numpy.real(numpy.dot(adjoint(psi),op.dot(psi)))
def norm_sq(psi):
    return numpy.real(numpy.dot(adjoint(psi),psi))
def normalize(psi,tol=1e-9):
    ns=norm_sq(psi)**0.5
    if ns < tol:
        raise ValueError
    return psi/ns
def is_herm(M,tol=1e-9):
    if M.shape[0]!=M.shape[1]:
        return False
    diff=M-adjoint(M)
    return max(numpy.abs(diff.flatten())) < tol
def is_unitary(M,tol=1e-9):
    if M.shape[0]!=M.shape[1]:
        return False
    diff=M.dot(adjoint(M))-numpy.identity((M.shape[0]))
    return max(numpy.abs(diff.flatten())) < tol
def eigu(U,tol=1e-9):
    (E_1,V_1)=numpy.linalg.eigh(U+adjoint(U))
    U_1=adjoint(V_1).dot(U).dot(V_1)
    H_1=adjoint(V_1).dot(U+adjoint(U)).dot(V_1)
    non_diag_lst=[]
    j=0
    while j < U_1.shape[0]:
        k=0
        while k < U_1.shape[0]:
            if j!=k and abs(U_1[j,k]) > tol:
                if j not in non_diag_lst:
                    non_diag_lst.append(j)
                if k not in non_diag_lst:
                    non_diag_lst.append(k)
            k+=1
        j+=1
    if len(non_diag_lst) > 0:
        non_diag_lst=numpy.sort(numpy.array(non_diag_lst))
        U_1_cut=U_1[non_diag_lst,:][:,non_diag_lst]
        (E_2_cut,V_2_cut)=numpy.linalg.eigh(1.j*(U_1_cut-adjoint(U_1_cut)))
        V_2=numpy.identity((U.shape[0]),dtype=V_2_cut.dtype)
        for j in range(len(non_diag_lst)):
            V_2[non_diag_lst[j],non_diag_lst]=V_2_cut[j,:]
        V_1=V_1.dot(V_2)
        U_1=adjoint(V_2).dot(U_1).dot(V_2)
    # Sort by phase
    U_1=numpy.diag(U_1)
    inds=numpy.argsort(numpy.imag(numpy.log(U_1)))
    return (U_1[inds],V_1[:,inds]) # = (U_d,V) s.t. U=V*U_d*V^\dagger
"""

py"""
f = open('exact_G_eigenvalues'+'.txt', 'w')
def Write_file(Eigenvalue):
    f = open('exact_G_eigenvalues'+'.txt', 'a')
    f.write(str(Eigenvalue) +'\n')
"""

In [128]:
#DELTA = 0.0
function MyEigenvalues(DELTA)
    List_of_U = [];
    List_of_H = [];
    Grover_matrix = sparse(Identity(2^L));
    Noise_Used = [];
    Noise_Counter = 1;
    
    #= U0 matrix. =#
    U0_XHL_Gates = []
    for i = 1:L-1
        push!(U0_XHL_Gates,["X",i])
    end    
    push!(U0_XHL_Gates,["H",L])

    U0_XHR_Gates = []
    for i = 1:L-1
        push!(U0_XHR_Gates,["X",i])
    end
    push!(U0_XHR_Gates,["H",L])
    
    MCX = sparse(Identity(2^L));
    
    XHL_Matrix = sparse(Identity(2^L))
    for i in U0_XHL_Gates
        if i[1] == "H"
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = Matrix_Gate(Hadamard(DELTA*epsilon), i[2]) 
            XHL_Matrix = XHL_Matrix*CRX
            push!(List_of_H,["H", DELTA*epsilon,i[2]])
            Noise_Counter += 1 
            
            # Without noise.
            CRX0 = Matrix_Gate(Hadamard(0.0), i[2])
            push!(List_of_U,CRX0)

        elseif i[1] == "X"
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = Matrix_Gate(CX(DELTA*epsilon),i[2])#Rx(pi+DELTA*epsilon)
            XHL_Matrix = XHL_Matrix*CRX
            push!(List_of_H,["X", DELTA*epsilon,i[2]])
            Noise_Counter += 1 
            
            # Without noise.
            CRX0 = Matrix_Gate(CX(0.0),i[2])
            push!(List_of_U,CRX0)
        end
    end
    
    #= Constructing the multicontrolled Toffoli gate. =# 
    # C_1.
    for i = 1:L-2
        for j = 1:i
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = CU(Rx((pi/2^j)+DELTA*epsilon), L-i, L-i+j)
            MCX = CRX*MCX
                
            push!(List_of_H,[pi/2^j+DELTA*epsilon, L-i, L-i+j])
            Noise_Counter += 1
                
            # Without noise.    
            CRX0 = CU(Rx((pi/2^j)), L-i, L-i+j)
            push!(List_of_U,CRX0)            
        end
    end

    # C_2.
    for i = 2:L

        epsilon = NOISE[Noise_Counter]
        push!(Noise_Used,epsilon)
        
        CRX = CU(Rx((pi/2^(i-2))+DELTA*epsilon), 1, i)
        MCX = CRX*MCX
            
        push!(List_of_H, [pi/2^(i-2)+DELTA*epsilon, 1, i])
        Noise_Counter += 1
            
        # Without noise.
        CRX0 = CU(Rx(pi/2^(i-2)), 1, i)
        push!(List_of_U,CRX0)
    end

    # C3 = - C1.
    for i = L-2:-1:1
        for j = i:-1:1
        
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = CU(Rx((-pi/2^j)+DELTA*epsilon), L-i, L-i+j)
            MCX = CRX*MCX
            
            push!(List_of_H,[-pi/2^j+DELTA*epsilon, L-i, L-i+j])
            Noise_Counter += 1
                
            # Without noise.    
            CRX0 = CU(Rx((-pi/2^j)), L-i, L-i+j)    
            push!(List_of_U,CRX0)
            
        end
    end

    # C_4.
    for i = 1:L-3
        for j = 1:i
        
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = CU(Rx((pi/2^j)+DELTA*epsilon), L-i-1, L-i-1+j)
            MCX = CRX*MCX
            
            push!(List_of_H,[pi/2^j+DELTA*epsilon,L-i-1, L-i-1+j])
            Noise_Counter += 1
                
            # Without noise.    
            CRX0 = CU(Rx((pi/2^j)), L-i-1, L-i-1+j)
            push!(List_of_U,CRX0)    
        end    
    end

    # C_5.
    for i = 2:L-1
        
        epsilon = NOISE[Noise_Counter]
        push!(Noise_Used,epsilon)
        
        CRX = CU(Rx((-pi/2^(i-2))+DELTA*epsilon), 1, i)
        MCX = CRX*MCX
        
        push!(List_of_H,[-pi/2^(i-2)+DELTA*epsilon,1, i])
        Noise_Counter += 1
            
        # Without noise.    
        CRX0 = CU(Rx((-pi/2^(i-2))), 1, i)    
        push!(List_of_U,CRX0)    
        
    end

    # C6 = - C4.
    for i = L-3:-1:1
        for j = i:-1:1
         
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = CU(Rx((-pi/2^j)+DELTA*epsilon), L-i-1, L-i-1+j)
            MCX = CRX*MCX
            
            push!(List_of_H,[-pi/2^j+DELTA*epsilon,L-i-1, L-i-1+j])
            Noise_Counter += 1    
                
            # Without noise.    
            CRX0 = CU(Rx((-pi/2^j)), L-i-1, L-i-1+j)    
            push!(List_of_U,CRX0)   
                
        end    
    end

    XHR_Matrix = sparse(Identity(2^L))
    for j in U0_XHR_Gates
        if j[1] == "H"
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = Matrix_Gate(Hadamard(DELTA*epsilon), j[2])
            XHR_Matrix = XHR_Matrix*CRX
            
            push!(List_of_H,["H", DELTA*epsilon,j[2]])
            Noise_Counter += 1 
            
            # Without noise.
            CRX0 = Matrix_Gate(Hadamard(0.0), j[2])    
            push!(List_of_U,CRX0) 
                
        elseif j[1] == "X"
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = Matrix_Gate(CX(DELTA*epsilon),j[2])
            XHR_Matrix = XHR_Matrix*CRX
            
            push!(List_of_H,["X", DELTA*epsilon,j[2]])
            Noise_Counter += 1 
                
            CRX0 = Matrix_Gate(CX(0.0),j[2])    
            push!(List_of_U,CRX0)    
        end
    end

    U0_matrix = sparse(XHL_Matrix*MCX*XHR_Matrix)    

    
    #= Ux matrix. =#
    Ux_XHL_Gates = []
    for i = 1:L-1
        push!(Ux_XHL_Gates,["H",i])
    end    
    for i = 1:L-1
        push!(Ux_XHL_Gates,["X",i])
    end  

    Ux_XHR_Gates = []
    for i = 1:L-1
        push!(Ux_XHR_Gates,["X",i])
    end    
    for i = 1:L-1
        push!(Ux_XHR_Gates,["H",i])
    end
    

    # Creating an empty matrix to store the Ux matrix.
    MCX = sparse(Identity(2^L));
    
    XHL_Matrix = sparse(Identity(2^L))
    for i in Ux_XHL_Gates
        
        if i[1] == "H"
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = Matrix_Gate(Hadamard(DELTA*epsilon), i[2])
            XHL_Matrix = XHL_Matrix*CRX
            
            push!(List_of_H,["H", DELTA*epsilon,i[2]])
            Noise_Counter += 1 
                
            # Without noise.    
            CRX0 = Matrix_Gate(Hadamard(0.0), i[2])    
            push!(List_of_U,CRX0)    
            
        elseif i[1] == "X"
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = Matrix_Gate(CX(DELTA*epsilon),i[2])
            XHL_Matrix = XHL_Matrix*CRX
            
            push!(List_of_H,["X", DELTA*epsilon,i[2]])
            Noise_Counter += 1 
                
            # Without noise.    
            CRX0 = Matrix_Gate(CX(0.0),i[2])    
            push!(List_of_U,CRX0)    
            
        end
    end
    
    #= Contructing the multicontrolled Toffoli gate. =#
    # C_1.
    for i = 1:L-2
        for j = 1:i
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = CU(Rx((pi/2^j)+DELTA*epsilon), L-i, L-i+j)
            MCX = CRX*MCX
            
            push!(List_of_H,[pi/2^j+DELTA*epsilon,L-i, L-i+j])
            Noise_Counter += 1 
                
            # Without noise.    
            CRX0 = CU(Rx((pi/2^j)), L-i, L-i+j)    
            push!(List_of_U,CRX0) 
                
        end
    end

    # C_2.
    for i = 2:L
        
        epsilon = NOISE[Noise_Counter]
        push!(Noise_Used,epsilon)
        
        CRX = CU(Rx((pi/2^(i-2))+DELTA*epsilon), 1, i)
        MCX = CRX*MCX
        
        push!(List_of_H,[pi/2^(i-2)+DELTA*epsilon, 1, i])
        Noise_Counter += 1
        
        # Without noise.    
        CRX0 = CU(Rx((pi/2^(i-2))), 1, i)    
        push!(List_of_U,CRX0)    
        
    end

    # C3 = - C1.
    for i = L-2:-1:1
        for j = i:-1:1
      
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = CU(Rx((-pi/2^j)+DELTA*epsilon), L-i, L-i+j)
            MCX = CRX*MCX
            
            push!(List_of_H,[-pi/2^j+DELTA*epsilon,L-i, L-i+j])
            Noise_Counter += 1
            
            # Without noise.    
            CRX0 = CU(Rx((-pi/2^j)), L-i, L-i+j)    
            push!(List_of_U,CRX0)    
        end
    end

    # C_4.
    for i = 1:L-3
        for j = 1:i
         
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = CU(Rx((pi/2^j)+DELTA*epsilon), L-i-1, L-i-1+j)
            MCX = CRX*MCX
            
            push!(List_of_H,[pi/2^j+DELTA*epsilon, L-i-1, L-i-1+j])
            Noise_Counter += 1
            
            # Without noise.    
            CRX0 = CU(Rx((pi/2^j)), L-i-1, L-i-1+j)    
            push!(List_of_U,CRX0)    
        end    
    end

    # C_5.
    for i = 2:L-1
      
        epsilon = NOISE[Noise_Counter]
        push!(Noise_Used,epsilon)
        
        CRX = CU(Rx((-pi/2^(i-2))+DELTA*epsilon), 1, i)
        MCX = CRX*MCX
        
        push!(List_of_H,[-pi/2^(i-2)+DELTA*epsilon,1, i])
        Noise_Counter += 1
        
        # Without noise.    
        CRX0 = CU(Rx((-pi/2^(i-2))), 1, i)    
        push!(List_of_U,CRX0)    
        
    end

    # C6 = - C4.
    for i = L-3:-1:1
        for j = i:-1:1
        
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = CU(Rx((-pi/2^j)+DELTA*epsilon), L-i-1, L-i-1+j)
            MCX = CRX*MCX
            
            push!(List_of_H,[-pi/2^j+DELTA*epsilon,L-i-1, L-i-1+j])
            Noise_Counter += 1 
                
            # Without noise.    
            CRX0 = CU(Rx((-pi/2^j)), L-i-1, L-i-1+j)    
            push!(List_of_U,CRX0)    
        end    
    end


    XHR_Matrix = sparse(Identity(2^L))
    for j in Ux_XHR_Gates
        if j[1] == "H"          
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            CRX = Matrix_Gate(Hadamard(DELTA*epsilon), j[2]) 
            XHR_Matrix = XHR_Matrix*CRX
            
            push!(List_of_H,["H", DELTA*epsilon,j[2]])
            Noise_Counter += 1  
            
            # Without noise.    
            CRX0 = Matrix_Gate(Hadamard(0.0), j[2])     
            push!(List_of_U,CRX0)  
                
        elseif j[1] == "X"         
            
            epsilon = NOISE[Noise_Counter]
            push!(Noise_Used,epsilon)
            
            CRX = Matrix_Gate(CX(DELTA*epsilon),j[2])
            XHR_Matrix = XHR_Matrix*CRX
            
            push!(List_of_H,["X", DELTA*epsilon,j[2]])
            Noise_Counter += 1 
                
            # Without noise.    
            CRX0 = Matrix_Gate(CX(0.0),j[2])    
            push!(List_of_U,CRX0)    
        end
    end
    
    Ux_matrix = sparse(XHL_Matrix*MCX*XHR_Matrix)   
    
    Grover_matrix = collect(Ux_matrix*U0_matrix)
    
    #= Diagonalizing Grover for exact eigenvalues. =#
    EIGU = py"eigu"(Grover_matrix)
    E_exact = real(1im*log.(EIGU[1])); # Eigenvalue.   
    E_exact = E_exact[2:2^L-1]; #= Neglecting the two special states at 1 and 2^L. =#
    

    #= Hs_list will contain all the gates H for the corresponding CRX,Hadamard and X in List_of_H. =#
    List_of_H = reverse(List_of_H)
    #List_of_U = reverse(List_of_U)
    function kth_term(k)
        
        f_k = Identity(2^L);
        for i = k:length(List_of_U)-1
            f_k = f_k * collect(List_of_U[length(List_of_U)-i+k])
        end 
        
        #= Corresponding H for the kth term. =#
        U_k = List_of_H[k]
        if U_k[1] == "H"
            Noise = U_k[2] # delta*epsilon.
            Qubit = U_k[3] # qubit.
            H_k = Matrix_Gate([1 0;0 1]-1/sqrt(2)*[1 1;1 -1],Qubit) #= H_had = I2-Had. =#
        elseif U_k[1] == "X"
            Noise = U_k[2] # delta*epsilon.
            Qubit = U_k[3] # qubit.
            H_k = Matrix_Gate([1 1;1 1],Qubit) #= H_X = X+I2. =#
 
        else
            Noise = U_k[1]
            Control_Qubit = int(U_k[2])
            Target_Qubit = int(U_k[3])
            #= H = ((I-Z)/2)_c \otimes ((I+X)/2)_t.=#
            Matrices = Dict("I" => I,"U" => (I-CX(0.0))/2, "PI_1" => (I-Z)/2)
            p1 = fill("I", L)
            p1[Control_Qubit] = "PI_1"
            p1[Target_Qubit] = "U"
            H_k = Matrices[p1[1]]
            for i = 2:L
                H_k = kron(H_k,Matrices[p1[i]])
            end                                 
        end
        return f_k * H_k * (f_k')
    end
    #= The following loop sums over all epsilon to get H_eff. =#
    h_eff = zeros(2^L,2^L);
    for ii = 1:length(List_of_U)
        h_eff += Noise_Used[ii]*kth_term(ii)
    end
    
    heff = DELTA*h_eff
    E_eff = eigen(heff[3:2^L,3:2^L]).values
    E_eff = sort(real(E_eff),rev = true)    
    return E_eff,E_exact 
    #return List_of_U
    #return U0_matrix,Ux_matrix
end;

In [129]:
#real(round.(collect(Ux_matrix*U0_matrix),digits=2))

### Verification of $\exp\{-i H \theta \} = CR_{x}(\theta)$ (WORKING)

In [130]:
#List_of_H[8]

In [131]:
#=
H3 = List_of_H[8] # CX
Noise = H3[1]
Control_Qubit = int(H3[2])
Target_Qubit = int(H3[3])
Matrices = Dict("I" => I,"U" => (I-CX(0.0))/2, "PI_1" => (I-Z)/2)
p1 = fill("I", L)
p1[Control_Qubit] = "PI_1"
p1[Target_Qubit] = "U"
H_k3 = Matrices[p1[1]]
for i = 2:L
    H_k3 = kron(H_k3,Matrices[p1[i]])
end 
exp(-1im*(H3[1])*H_k3)=#

### Exact Grover operator

In [132]:
U_0 = [-1 0 0 0; 0 1 0 0; 0 0 1 0;0 0 0 1];
A = ones(2^L,2^L);
U_x = (2/2^L)*A-Identity(2^L); # 2\s><s|-I
G_exact = U_x*U_0
#U_x

4×4 Matrix{Float64}:
  0.5   0.5   0.5   0.5
 -0.5  -0.5   0.5   0.5
 -0.5   0.5  -0.5   0.5
 -0.5   0.5   0.5  -0.5

### Grover by multiplying matrices of List_of_U (WORKING)

In [133]:
#G = -real(round.(List_of_U[10]*List_of_U[9]*List_of_U[8]*List_of_U[7]*List_of_U[6]*List_of_U[5]*List_of_U[4]*List_of_U[3]*List_of_U[2]*List_of_U[1],digits=2))

### Grover by multiplying matrices of List_of_H

In [134]:
#=
G_from_U = Identity(2^L);
G_from_U_list = []
for U_k in reverse(List_of_H) # reverse is necessary.
        if U_k[1] == "H"
            Noise = U_k[2] # delta*epsilon.
            Qubit = U_k[3] # qubit.
            H_k = Matrix_Gate([1 0;0 1]-1/sqrt(2)*[1 1;1 -1],Qubit) #= H_had = I2-Had. =#
            G_from_U = G_from_U*exp(-1im*(pi/2+Noise)*collect(H_k))
            push!(G_from_U_list,exp(-1im*(pi/2+Noise)*collect(H_k)))
        elseif U_k[1] == "X"
            Noise = U_k[2] # delta*epsilon.
            Qubit = U_k[3] # qubit.
            H_k = Matrix_Gate([1 1;1 1],Qubit) #= H_X = X+I2. =#
            G_from_U = G_from_U*exp(-1im*(pi/2+Noise)*collect(H_k))
            push!(G_from_U_list,exp(-1im*(pi/2+Noise)*collect(H_k)))
        else
            Angle = U_k[1]
            Control_Qubit = int(U_k[2])
            Target_Qubit = int(U_k[3])
            #= H = ((I-Z)/2)_c \otimes ((I+X)/2)_t.=#
            Matrices = Dict("I" => I,"U" => (I-CX(0.0))/2, "PI_1" => (I-Z)/2)
            p1 = fill("I", L)
            p1[Control_Qubit] = "PI_1"
            p1[Target_Qubit] = "U"
            H_k = Matrices[p1[1]]
            for i = 2:L
                H_k = kron(H_k,Matrices[p1[i]]);
            end   
            G_from_U = G_from_U*exp(-1im*(Angle)*collect(H_k))
            push!(G_from_U_list,exp(-1im*(Angle)*collect(H_k)))
        end
end
real(round.(-G_from_U,digits=2))=#

In [135]:
G_exact

4×4 Matrix{Float64}:
  0.5   0.5   0.5   0.5
 -0.5  -0.5   0.5   0.5
 -0.5   0.5  -0.5   0.5
 -0.5   0.5   0.5  -0.5

In [136]:
#=
k = 1
    f_k = Identity(2^L);
        for i = k:length(List_of_U)-1
            f_k = f_k * collect(List_of_U[length(List_of_U)-i+k])
        end 
        
        #= Corresponding H for the kth term. =#
        U_k = List_of_H[k]
        if U_k[1] == "H"
            Noise = U_k[2] # delta*epsilon.
            Qubit = U_k[3] # qubit.
            H_k = Matrix_Gate(I2-H,Qubit) #= H_had = I2-Had. =#
        elseif U_k[1] == "X"
            Noise = U_k[2] # delta*epsilon.
            Qubit = U_k[3] # qubit.
            H_k = Matrix_Gate(I+CX(0.0),Qubit) #= H_X = X+I2. =#
 
        else
            Noise = U_k[1]
            Control_Qubit = int(U_k[2])
            Target_Qubit = int(U_k[3])
            #= H = ((I-Z)/2)_c \otimes ((I+X)/2)_t.=#
            Matrices = Dict("I" => I,"U" => (I+CX(0.0))/2, "PI_1" => (I-Z)/2)
            p1 = fill("I", L)
            p1[Control_Qubit] = "PI_1"
            p1[Target_Qubit] = "U"
            H_k = Matrices[p1[1]]
            for i = 2:L
                H_k = kron(H_k,Matrices[p1[i]])
            end                                 
        end
K1 = real(round.(f_k*H_k*(f_k'),digits=2))
#real(round.(H_k,digits=2))
#real(round.(H_k,digits=2))=#

### All h_eff terms

In [137]:
#=
# 10 th term.
H10 = List_of_H[10] # H
Noise = H10[2] # delta*epsilon.
Qubit10 = H10[3] # qubit.
H_k10 = collect(Matrix_Gate(I-H,Qubit10))
# 9 th term.
H9 = List_of_H[9] # X
Noise = H9[2] # delta*epsilon.
Qubit9 = H9[3] # qubit.
H_k9 = List_of_U[10]*Matrix_Gate(-[0 1;1 0]+I,Qubit9)*(List_of_U[10])'
real(round.(H_k9,digits=2))
# 8 th term.
H8 = List_of_H[8] # CX
Noise = H8[1]
Control_Qubit = int(H8[2])
Target_Qubit = int(H8[3])
Matrices = Dict("I" => I,"U" => (I+CX(0.0))/2, "PI_1" => (I-Z)/2)
p1 = fill("I", L)
p1[Control_Qubit] = "PI_1"
p1[Target_Qubit] = "U"
H_k8 = Matrices[p1[1]]
for i = 2:L
    H_k8 = kron(H_k8,Matrices[p1[i]])
end 
H_k8 = List_of_U[10]*List_of_U[9]*H_k8*List_of_U[9]'*List_of_U[10]'
real(round.(H_k8,digits=2))
# 7 th term
H7 = List_of_H[7] # X
Noise = H7[2] # delta*epsilon.
Qubit7 = H7[3] # qubit.
H_k7 = List_of_U[10]*List_of_U[9]*List_of_U[8]*Matrix_Gate(CX(0.0)+I,Qubit7)*List_of_U[8]'*List_of_U[9]'*List_of_U[10]'
real(round.(H_k7,digits=2))
# 6 th term.
H6 = List_of_H[6] # H
Noise = H6[2] # delta*epsilon.
Qubit6 = H6[3] # qubit.
H_k6 = List_of_U[10]*List_of_U[9]*List_of_U[8]*List_of_U[7]*Matrix_Gate(I-H,Qubit6)*List_of_U[7]'*List_of_U[8]'*List_of_U[9]'*List_of_U[10]'
real(round.(H_k6,digits=2))
# 5 th term.
H5 = List_of_H[5] # H
Noise = H5[2] # delta*epsilon.
Qubit5 = H5[3] # qubit.
H_k5 = List_of_U[10]*List_of_U[9]*List_of_U[8]*List_of_U[7]*List_of_U[6]*Matrix_Gate(I-H,Qubit5)*List_of_U[6]'*List_of_U[7]'*List_of_U[8]'*List_of_U[9]'*List_of_U[10]'
real(round.(H_k5,digits=2))
# 4 th term
H4 = List_of_H[4] # X
Noise = H4[2] # delta*epsilon.
Qubit4 = H4[3] # qubit.
H_k4 = List_of_U[10]*List_of_U[9]*List_of_U[8]*List_of_U[7]*List_of_U[6]*List_of_U[5]*
Matrix_Gate(CX(0.0)+I,Qubit4)*List_of_U[5]*List_of_U[6]*List_of_U[7]'*List_of_U[8]'*List_of_U[9]'*List_of_U[10]'
real(round.(H_k4,digits=2))
# 3 rd term
H3 = List_of_H[3] # CX
Noise = H3[1]
Control_Qubit = int(H3[2])
Target_Qubit = int(H3[3])
Matrices = Dict("I" => I,"U" => (I+CX(0.0))/2, "PI_1" => (I-Z)/2)
p1 = fill("I", L)
p1[Control_Qubit] = "PI_1"
p1[Target_Qubit] = "U"
H_k3 = Matrices[p1[1]]
for i = 2:L
    H_k3 = kron(H_k3,Matrices[p1[i]])
end 
H_k3 = List_of_U[10]*List_of_U[9]*List_of_U[8]*List_of_U[7]*List_of_U[6]*List_of_U[5]*List_of_U[4]*H_k3*List_of_U[4]'*List_of_U[5]'*List_of_U[6]'*List_of_U[7]'*List_of_U[8]'*List_of_U[9]'*List_of_U[10]'
real(round.(H_k3,digits=2))
# 2 nd term
H2 = List_of_H[2] # H
Noise = H2[2] # delta*epsilon.
Qubit2 = H2[3] # qubit.
H_k2 = List_of_U[10]*List_of_U[9]*List_of_U[8]*List_of_U[7]*List_of_U[6]*List_of_U[5]*List_of_U[4]*List_of_U[3]*Matrix_Gate(I-H,Qubit2)*List_of_U[3]'*List_of_U[4]'*List_of_U[5]'*List_of_U[6]'*List_of_U[7]'*List_of_U[8]'*List_of_U[9]'*List_of_U[10]'
real(round.(H_k2,digits=2))
# 1 st term
H1 = List_of_H[1] # X
Noise = H1[2] # delta*epsilon.
Qubit1 = H1[3] # qubit.
H_k1 = List_of_U[10]*List_of_U[9]*List_of_U[8]*List_of_U[7]*List_of_U[6]*List_of_U[5]*List_of_U[4]*List_of_U[3]*List_of_U[2]*Matrix_Gate(CX(0.0)+I,Qubit1)*List_of_U[2]'*List_of_U[3]'*List_of_U[4]'*List_of_U[5]'*List_of_U[6]'*List_of_U[7]'*List_of_U[8]'*List_of_U[9]'*List_of_U[10]'
real(round.(H_k1,digits=2))=#

### Hks

In [138]:
#=
Hks = [H_k10,H_k9,H_k8,H_k7,H_k6,H_k5,H_k4,H_k3,H_k2,H_k1];
Zz = zeros(2^L,2^L)
h_effective = Noise_Used[10]*H_k10+Noise_Used[9]*H_k9+Noise_Used[8]*H_k8+Noise_Used[7]*H_k7+Noise_Used[6]*H_k6+Noise_Used[5]*H_k5+Noise_Used[4]*H_k4+Noise_Used[3]*H_k3+Noise_Used[2]*H_k2+Noise_Used[1]*H_k1
h_effective = DELTA*h_effective
E_effective = eigen(h_effective[3:2^L,3:2^L]).values
E_effective  = sort(real(E_effective),rev = true)=#

In [139]:
py"""
f = open('new_eigenvalues_data'+'.txt', 'w')
def Write_file2(delta, effective, exact):
    f = open('new_eigenvalues_data'+'.txt', 'a')
    f.write(str(delta) + '\t' + str(effective)+ '\t' + str(exact) +'\n')
"""
Num = 100;
for i = 1:Num
    delta = 0.1*(i/Num)
    EE = MyEigenvalues(delta);
    effective = EE[1]
    exact = EE[2]
    for j = 1:2^L-2
        py"Write_file2"(delta,effective[j],exact[j])
        #println(effective[j])
        #println(exact[j])
    end
end